In [1]:
import pandas as pd
from math import log10
data = pd.read_csv("../data/data.csv", encoding='utf-8',delimiter=";")

SENW = {'S': 0, 'E': 1, 'N': 2, 'W': 3}
LP = {'l': 0, 'p': 1}
ETAGES = {'-1': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7}
macs = {}

def res2df(x, i):
    a = x.split('|')
    a[0] = LP[a[0]]
    a[1] = SENW[a[1]]
    a[2] = ETAGES[a[2]]
    return int(a[i])

def mac2id(x):
    if macs.get(x):
        return macs[x]
    macs[x] = len(macs)
    return int(macs[x])

for x in range (0, 10):
    data['mac'+str(x)] = data['mac'+str(x)].apply(lambda x: mac2id(x))
    data['dbm'+str(x)] = 10 ** (( 27.55 - (20 * log10(2400)) + data['dbm'+str(x)] ) / 20 )
    data['ssid'+str(x)] = data['ssid'+str(x)].apply(lambda x: mac2id(x))
#    data['X'] = data
data['res0'] = data['res'].dropna().apply(lambda x: res2df(x, 0))
data['res1'] = data['res'].dropna().apply(lambda x: res2df(x, 1))
data['res2'] = data['res'].dropna().apply(lambda x: int(res2df(x, 2)))

target = [
    ['l', 'p'],
    ['S', 'E', 'N', 'W'],
    ['-1', '0', '1', '2', '3', '4', '5', '6'], 
    [''] * 200
]


In [2]:
datalearn = data[data['res'].isna() == False]
datalearn

,res,dbm0,mac0,ssid0,dbm1,mac1,ssid1,dbm2,mac2,ssid2,...,dbm8,mac8,ssid8,dbm9,mac9,ssid9,Unnamed: 31,res0,res1,res2
123,p|W|4,3.526052,4,44,9.937766,7,44,15.750297,58,45,...,55.884164,8,44,62.703063,2,44,NaN,1.0,3.0,5.0
124,p|W|4,3.956296,4,44,9.937766,7,44,17.672124,3,45,...,44.390369,8,44,55.884164,2,44,NaN,1.0,3.0,5.0
125,p|S|6,1.982845,6,44,4.439037,5,44,7.893848,58,45,...,88.570431,107,113,111.503566,138,125,NaN,1.0,0.0,7.0
126,p|S|6,1.767212,6,44,6.270306,5,44,15.750297,63,44,...,99.377658,129,134,140.374673,138,125,NaN,1.0,0.0,7.0
127,p|S|6,1.403747,6,44,7.035399,5,44,7.893848,63,44,...,78.938480,118,125,99.377658,30,44,NaN,1.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,l|E|4,3.956296,1,44,11.150357,31,45,11.150357,28,44,...,55.884164,41,57,62.703063,108,95,NaN,0.0,1.0,5.0
262,l|N|4,11.150357,1,44,12.510906,31,45,22.247886,28,44,...,280.084295,39,55,280.084295,38,54,NaN,0.0,2.0,5.0
263,l|E|4,7.035399,2,44,17.672124,1,44,19.828450,3,45,...,99.377658,14,47,111.503566,7,44,NaN,0.0,1.0,5.0
264,l|E|4,5.588416,2,44,8.857043,1,44,19.828450,3,45,...,55.884164,27,44,70.353994,40,56,NaN,0.0,1.0,5.0


In [3]:
import numpy as np

X = datalearn[['dbm0', 'mac0']].values
y0 = datalearn[['res0']].values
y1 = datalearn[['res1']].values
y2 = datalearn[['res2']].values
for i in range(1, 10): 
    nan_dbm = datalearn['dbm'+str(i)].isna() == False
    nan_mac = datalearn['mac'+str(i)].isna() == False
    nan = nan_dbm & nan_mac
    X = np.append(X, datalearn[['dbm'+str(i), 'mac'+str(i)]][nan].values, axis = 0)
    y0 = np.append(y0, datalearn[['res0']][nan].values, axis = 0)
    y1 = np.append(y1, datalearn[['res1']][nan].values, axis = 0)
    y2 = np.append(y2, datalearn[['res2']][nan].values, axis = 0)

#len(X)

X = pd.DataFrame(X)
y0 = pd.DataFrame(y0)
y1 = pd.DataFrame(y1)
y2 = pd.DataFrame(y2)


In [4]:
from sklearn.ensemble import GradientBoostingClassifier

lr0 = GradientBoostingClassifier(random_state=1, n_estimators=100)
lr0 = lr0.fit(X, y0.values.ravel())
new_learn_score0 = lr0.score(X, y0)
lr1 = GradientBoostingClassifier()
lr1 = lr1.fit(X, y1.values.ravel())
new_learn_score1 = lr1.score(X, y1)
lr2 = GradientBoostingClassifier()
lr2 = lr2.fit(X, y2.values.ravel())
new_learn_score2 = lr2.score(X, y2)


[new_learn_score0, new_learn_score1, new_learn_score2]

[0.8436599423631124, 0.6217579250720461, 0.5994236311239193]

In [5]:
predicted = []

for mesure in datalearn.values:
    data2predict = []
    for i in range (0, 10):
        x = mesure[i*3 + 1 : i*3 + 3]
        if x[0] > 0:
            data2predict.append(x)
    
    predicted.append([
        pd.DataFrame(pd.DataFrame(lr0.predict_proba(data2predict)).mean()).sort_values(by=0, ascending=False).index[0],
        pd.DataFrame(pd.DataFrame(lr1.predict_proba(data2predict)).mean()).sort_values(by=0, ascending=False).index[0],
        pd.DataFrame(pd.DataFrame(lr2.predict_proba(data2predict)).mean()).sort_values(by=0, ascending=False).index[0],
    ])
predicted = pd.DataFrame(predicted)
predicted

,0,1,2
0,1,0,5
1,1,0,5
2,1,0,7
3,1,0,7
4,1,0,7
...,...,...,...
138,0,1,5
139,1,2,5
140,0,1,5
141,0,1,5


In [6]:
origin = pd.DataFrame(datalearn[['res0', 'res1', 'res2']].values)
origin

,0,1,2
0,1.0,3.0,5.0
1,1.0,3.0,5.0
2,1.0,0.0,7.0
3,1.0,0.0,7.0
4,1.0,0.0,7.0
...,...,...,...
138,0.0,1.0,5.0
139,0.0,2.0,5.0
140,0.0,1.0,5.0
141,0.0,1.0,5.0


In [9]:
from sklearn.metrics import classification_report

origin[3] = origin[2] + origin[1]*10 + origin[0]*100
predicted[3] = predicted[2] + predicted[1]*10 + predicted[0]*100

for i in range(0,3):
    print(classification_report(origin[i], predicted[i], target_names=target[i]))

res = classification_report(origin[3], predicted[3], output_dict = True)
t = []
for i in res.keys():
    try:
        i = "{:03d}".format(int(float(i)))
        t.append(target[0][int(i[0])]+'|'+target[1][int(i[1])]+'|'+target[2][int(i[2])])
    except:
        continue
print(classification_report(origin[3], predicted[3], target_names=t))

              precision    recall  f1-score   support

           l       0.97      0.90      0.93        63
           p       0.93      0.97      0.95        80

    accuracy                           0.94       143
   macro avg       0.95      0.94      0.94       143
weighted avg       0.95      0.94      0.94       143

              precision    recall  f1-score   support

           S       0.84      0.93      0.88        41
           E       1.00      0.33      0.50        15
           N       0.70      0.98      0.82        51
           W       0.95      0.58      0.72        36

    accuracy                           0.80       143
   macro avg       0.88      0.71      0.73       143
weighted avg       0.84      0.80      0.78       143

              precision    recall  f1-score   support

          -1       1.00      0.75      0.86        16
           0       0.81      1.00      0.90        26
           1       0.85      0.92      0.88        24
           2       1.

In [10]:
import joblib

tobesaved = [lr0, lr1, lr2, LP, SENW, ETAGES, macs]
files = joblib.dump(tobesaved, '../data/learner.lr')
